<a href="https://colab.research.google.com/github/jdmartinezrs/MachineLearningAlgoritms/blob/main/mision3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
sheet_id = '1Te2gVFrIhMKCPS5WDFTz7XuLQBBG_8CJ'
sheet_url = f'https://drive.google.com/uc?id={sheet_id}&export=download'
df = pd.read_excel(sheet_url, header=5)

Limpieza de datos

In [35]:
df.rename(columns={
    'Ult Km/Hr': 'Ult_Km',
    'Km/ Hr Plan': 'Km_Plan',
    'Km/Hr Cambio': 'Km_Cambio',
    'Dias Pend': 'Dias_Pend',
    'Tipo de Vehiculo': 'Tipo_Vehiculo',
    'Km/Hr Pend': 'Km_Pend',
    'fecha de revision': 'fecha_revision'
}, inplace=True)

In [31]:
# Lista de columnas numéricas en las que queremos corregir el formato
columnas_numericas = ['Ult_Km', 'Km_Plan', 'Km_Cambio', 'Dias_Pend', 'Km_Pend']

# Reemplazar comas por puntos en las columnas numéricas y convertir a tipo float
for col in columnas_numericas:
    df[col] = df[col].astype(str).str.replace(',', '.').astype(float)

In [39]:
# Eliminar filas con valores faltantes (NaN)
df_sin_nulos = df.dropna()
print("\nSin nulos:\n", df_sin_nulos)
# Rellenar valores faltantes con la media de cada columna numérica
df_filled = df.copy()   #Crea una copia independiente del DataFrame original (df) y la guarda como df_filled.
df_filled['Ult_Km'] = df_filled['Ult_Km'].fillna(df['Ult_Km'].mean())
df_filled['Km_Plan'] = df_filled['Km_Plan'].fillna(df['Km_Plan'].mean())
df_filled['Km_Cambio'] = df_filled['Km_Cambio'].fillna(df['Km_Cambio'].mean())
df_filled['Dias_Pend'] = df_filled['Dias_Pend'].fillna(df['Dias_Pend'].mean())
df_filled['Km_Pend'] = df_filled['Km_Pend'].fillna(df['Km_Pend'].mean())


Sin nulos:
                      Fecha  ITEM CODIGO DE EQUIPO Tipo_Vehiculo  \
19 2025-05-23 17:24:11.232  20.0  VE-GR-FO-WFG316          GRUA   

   fecha_revision  Km_Plan Fecha Ult Pre-Operacional  Dias_Pend    Ult_Km  \
19     2025-05-23   5000.0                2025-05-20        3.0  149068.0   

    Km_Cambio Fecha Cambio Fecha Prox Cambio Unnamed: 12  Km/Hr Proyectada  \
19   146837.0   2025-01-20        2025-05-10  2025-12-06          151837.0   

    Km_Pend  Progreso  Unnamed: 16 Observaciones   
19   2769.0    0.5538        27.69      trienergy  


In [40]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Supongamos que 'df' es tu DataFrame actual

# Columnas numéricas a escalar
columnas_a_escalar = ['Ult_Km', 'Km_Plan', 'Km_Cambio', 'Dias_Pend', 'Km_Pend']

# Columna categórica a aplicar One-Hot Encoding
columna_categorica = ['Tipo_Vehiculo']

# Columna de fecha (si quieres extraer características numéricas)
# Para este ejemplo, vamos a convertirla a datetime y luego extraer el año
df['fecha_revision'] = pd.to_datetime(df['fecha_revision'], format='%d/%m/%Y')
df['año_revision'] = df['fecha_revision'].dt.year
columnas_fecha_num = ['año_revision'] # Agregamos el año como columna numérica

# Definir el preprocesamiento para cada tipo de columna
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), columnas_a_escalar + columnas_fecha_num), # Escalar columnas numéricas y la columna numérica extraída de la fecha
        ('cat', OneHotEncoder(), columna_categorica) # Aplicar One-Hot Encoding a la columna categórica
    ],
    remainder='passthrough' # Mantener las columnas no especificadas (como la columna original de fecha si no la quieres eliminar)
)

# Crear un pipeline para aplicar el preprocesamiento
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Aplicar el preprocesamiento al DataFrame
df_procesado = pipeline.fit_transform(df)

# 'df_procesado' es ahora un array NumPy con tus datos transformados
# Si quieres convertirlo de vuelta a un DataFrame, necesitarías manejar los nombres de las columnas
# generadas por OneHotEncoder, lo cual puede ser más complejo.
# Para empezar, puedes trabajar con el array NumPy.

In [41]:
print(df_procesado)

[[0.3072877347589628 0.7142857142857143 0.3042025298845389 0.0
  0.4832955167827243 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 NaT 1.0
  'VE-DT-CH-XMD284' Timestamp('2025-05-23 00:00:00')
  Timestamp('2025-05-20 00:00:00') Timestamp('2024-06-21 00:00:00')
  Timestamp('2025-01-17 00:00:00') Timestamp('2025-08-15 00:00:00')
  326580.0 0.5748 57.48 57.48]
 [0.2001257568521115 0.7142857142857143 0.19142722161947004
  0.9999999999999999 0.05758547844456615 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0
  0.0 NaT 2.0 'VE-DT-FL-XMD588' Timestamp('2025-05-23 00:00:00')
  Timestamp('2023-09-21 00:00:00') Timestamp('2023-07-11 00:00:00') NaT
  NaT 209850.0 0.0307 nan nan]
 [0.41418105905372304 0.4285714285714287 0.41359931289472085 0.0
  0.5007432908223143 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 NaT 3.0
  'VE-DT-FL-SUE755' Timestamp('2025-05-23 00:00:00')
  Timestamp('2025-05-20 00:00:00') Timestamp('2025-03-12 00:00:00')
  Timestamp('2025-10-08 00:00:00') Timestamp('2026-05-06 00:00:00')
  437813.0 0.746375 59.71 nan]
 

In [50]:
# Get the column names after one-hot encoding
# This requires knowing the categories in your categorical column
# You can get the categories from the preprocessor
ohe_categories = preprocessor.named_transformers_['cat'].categories_[0]
ohe_column_names = [f'Tipo_Vehiculo_{cat}' for cat in ohe_categories]

# Get the names of the columns that were passed through
# You'll need to identify these columns from your original DataFrame
# For this example, let's assume the original date columns and other columns are passed through
passthrough_columns = [col for col in df.columns if col not in columnas_a_escalar + columnas_fecha_num + columna_categorica]

# Combine all the column names in the correct order
all_processed_columns = columnas_a_escalar + columnas_fecha_num + ohe_column_names + passthrough_columns

# Create a new DataFrame from the processed array with the combined column names
df_procesado_df = pd.DataFrame(df_procesado, columns=all_processed_columns)

# Display the new DataFrame
print(df_procesado_df.head())

     Ult_Km   Km_Plan Km_Cambio Dias_Pend   Km_Pend año_revision  \
0  0.307288  0.714286  0.304203       0.0  0.483296          0.0   
1  0.200126  0.714286  0.191427       1.0  0.057585          0.0   
2  0.414181  0.428571  0.413599       0.0  0.500743          0.0   
3  0.258086  0.714286  0.253086       0.0  0.341523          0.0   
4  0.276062  0.714286  0.276525       0.0  0.779047          0.0   

  Tipo_Vehiculo_CARRO TANQUE Tipo_Vehiculo_DOBLETROQUE Tipo_Vehiculo_GRUA  \
0                        1.0                       0.0                0.0   
1                        0.0                       1.0                0.0   
2                        0.0                       1.0                0.0   
3                        0.0                       1.0                0.0   
4                        0.0                       1.0                0.0   

  Tipo_Vehiculo_IRRIGADOR  ... CODIGO DE EQUIPO fecha_revision  \
0                     0.0  ...  VE-DT-CH-XMD284     2025-05-23

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Seleccionar las columnas numéricas a escalar (basado en tu código anterior)
columnas_a_escalar = ['Ult_Km', 'Km_Plan', 'Km_Cambio', 'Dias_Pend', 'Km_Pend']

# Crear una instancia de MinMaxScaler
scaler = MinMaxScaler()

# Aplicar MinMaxScaler a las columnas seleccionadas
df[columnas_a_escalar] = scaler.fit_transform(df[columnas_a_escalar])

# Ahora las columnas seleccionadas en tu DataFrame 'df' están escaladas entre 0 y 1